# 01_dataset_schema_exploration.ipynb
# 📡 Satellite Communications Dataset – Schema Exploration

"""
Overview:
This notebook explores the SatNOGS satellite communications dataset schema.
Goal: Understand the structure of all tables, their columns, and row counts,
so we can later prepare the dataset for machine learning tasks.

Objectives:
- Inspect each table in the database (`satnogs`)
- Document column names and row counts
- Preview sample rows for context
- Identify relationships between tables
- Save schema summaries for reuse
"""

In [2]:
# --- Install dependencies (only needed once in environment) ---
%pip install sqlalchemy pandas pymysql tabulate
%pip install tabulate
import sqlalchemy
import pandas as pd
import importlib
from sqlalchemy import create_engine, inspect

print("SQLAlchemy version:", sqlalchemy.__version__)
print("Pandas version:", pd.__version__)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
SQLAlchemy version: 2.0.44
Pandas version: 2.3.3


In [3]:
# --- Database connection ---
DB_USER = "root"
DB_PASSWORD = "123456789"
DB_HOST = "127.0.0.1"
DB_PORT = "3306"
DB_NAME = "satnogs"

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
inspector = inspect(engine)


In [4]:
# --- List all tables ---
tables = inspector.get_table_names()
print("Tables in database:", tables)


Tables in database: ['base_antenna', 'base_antennatype', 'base_frequencyrange', 'base_launch', 'base_mode', 'base_observation', 'base_operator', 'base_satellite', 'base_satelliteentry', 'base_satelliteidentifier', 'base_station', 'base_stationstatuslog', 'base_stationtype', 'base_telemetry', 'base_transmitterentry']


In [9]:
# --- Check the dataset timeline ---

import time

def run_query(sql, engine, desc="Query"):
    """Run SQL query and return DataFrame with logging."""
    print(f"\n🔍 Running: {desc}")
    start = time.time()
    df = pd.read_sql_query(sql, engine)
    print(f"✅ Retrieved {len(df):,} rows in {time.time()-start:.2f}s")
    return df
timeline_sql = """
SELECT MIN(start) AS first_observation,
       MAX(start) AS last_observation
FROM base_observation
WHERE start IS NOT NULL
"""
timeline = run_query(timeline_sql, engine, "Dataset timeline")
print(timeline)


🔍 Running: Dataset timeline
✅ Retrieved 1 rows in 30.95s
    first_observation    last_observation
0 2015-10-12 15:13:16 2025-11-12 09:59:41


In [10]:
# --- Helper function to explore a table ---
def explore_table(table_name, preview_rows=5):
    """Return schema summary for a given table."""
    cols = [c['name'] for c in inspector.get_columns(table_name)]
    count = pd.read_sql(f"SELECT COUNT(*) AS total_rows FROM {table_name}", engine)['total_rows'][0]
    df_preview = pd.read_sql(f"SELECT * FROM {table_name} LIMIT {preview_rows}", engine)
    
    summary = {
        "table": table_name,
        "columns": cols,
        "row_count": count,
        "preview": df_preview
    }
    return summary

In [11]:
#  --- Explore all relevant tables ---
tables_to_explore = [
    "base_antenna",
    "base_antennatype",
    "base_frequencyrange",
    "base_launch",
    "base_mode",
    "base_observation",
    "base_operator",
    "base_satellite",
    "base_satelliteentry",
    "base_satelliteidentifier",
    "base_station",
    "base_stationstatuslog",
    "base_stationtype",
    "base_telemetry",
    "base_transmitterentry"
]

schema_summaries = []
for tbl in tables_to_explore:
    summary = explore_table(tbl)
    schema_summaries.append(summary)
    print(f"\n=== {tbl} ===")
    print("Columns:", summary["columns"])
    print("Row count:", summary["row_count"])
    print(summary["preview"].head())
    print(f"✅ Finished processing {tbl}")


=== base_antenna ===
Columns: ['id', 'antenna_type_id', 'station_id']
Row count: 4437
   id  antenna_type_id  station_id
0   1               12         256
1   2               12        1536
2   7               10         771
3   8               13        1285
4   9               15           6
✅ Finished processing base_antenna

=== base_antennatype ===
Columns: ['id', 'name']
Row count: 17
   id          name
0   6    Cross Yagi
1   1        Dipole
2   3       Discone
3  12     Eggbeater
4   4  Ground Plane
✅ Finished processing base_antennatype

=== base_frequencyrange ===
Columns: ['id', 'min_frequency', 'max_frequency', 'antenna_id']
Row count: 5311
   id  min_frequency  max_frequency  antenna_id
0   1      135000000      148000000           1
1   2      430000000      440000000           2
2   7      135000000      148000000           7
3   8      430000000      440000000           8
4   9      400000000      470000000           9
✅ Finished processing base_frequencyrange

=== b

In [12]:
# --- Enhanced NULL analysis for all tables ---
def detailed_null_analysis(table_name, sample_size=10000):
    """Perform comprehensive NULL analysis on a table"""
    print(f"\n{'='*60}")
    print(f"NULL Analysis for: {table_name}")
    print('='*60)
    
    # Get total row count
    total_count = pd.read_sql(f"SELECT COUNT(*) as total FROM {table_name}", engine)['total'][0]
    print(f"Total rows: {total_count:,}")
    
    # Sample data for analysis (if table is large)
    df_sample = pd.read_sql(f"SELECT * FROM {table_name} LIMIT {sample_size}", engine)
    print(f"Sample size for analysis: {len(df_sample):,}")
    
    # Calculate NULL percentages
    null_stats = (df_sample.isnull().sum() / len(df_sample) * 100).sort_values(ascending=False)
    
    # Create summary DataFrame
    null_df = pd.DataFrame({
        'column': null_stats.index,
        'null_percentage': null_stats.values,
        'non_null_count': (len(df_sample) - df_sample.isnull().sum()).values
    })
    
    # Display columns with NULLs
    columns_with_nulls = null_df[null_df['null_percentage'] > 0]
    
    print(f"\nColumns with NULL values ({len(columns_with_nulls)} out of {len(df_sample.columns)}):")
    for _, row in columns_with_nulls.head(15).iterrows():
        print(f"  • {row['column']:40} - {row['null_percentage']:6.2f}% NULL ({row['non_null_count']:,} non-NULL)")
    
    if len(columns_with_nulls) > 15:
        print(f"  ... and {len(columns_with_nulls) - 15} more columns with NULLs")
    
    return null_df

# Analyze key tables
key_tables = ['base_observation', 'base_station', 'base_satelliteentry']
for table in key_tables:
    detailed_null_analysis(table)


NULL Analysis for: base_observation
Total rows: 12,546,241
Sample size for analysis: 10,000

Columns with NULL values (20 out of 48):
  • station_antennas                         - 100.00% NULL (10,000 non-NULL)
  • station_lat                              - 100.00% NULL (10,000 non-NULL)
  • station_alt                              - 100.00% NULL (10,000 non-NULL)
  • station_lng                              - 100.00% NULL (10,000 non-NULL)
  • center_frequency                         - 100.00% NULL (9,165 non-NULL)
  • transmitter_status                       - 100.00% NULL (8,757 non-NULL)
  • transmitter_unconfirmed                  - 100.00% NULL (8,757 non-NULL)
  • transmitter_uplink_drift                 - 100.00% NULL (8,757 non-NULL)
  • transmitter_downlink_drift               -  99.98% NULL (10,000 non-NULL)
  • transmitter_uplink_high                  -  99.91% NULL (10,000 non-NULL)
  • transmitter_downlink_high                -  99.91% NULL (10,000 non-NULL)
  • transmi

In [13]:
# --- Deep dive into observation status ---
print("\n" + "="*60)
print("DEEP DIVE: OBSERVATION STATUS ANALYSIS")
print("="*60)

# Get complete status distribution
complete_status = pd.read_sql("""
    SELECT 
        status,
        waterfall_status,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM base_observation 
    GROUP BY status, waterfall_status
    ORDER BY count DESC
    LIMIT 15
""", engine)

print("Top 15 Status Combinations:")
print(complete_status.to_string(index=False))

# Interpret the status codes (based on your findings)
print("\n" + "-"*40)
print("STATUS CODE INTERPRETATION (Hypothesis):")
print("-"*40)
print("status = 100:    Observation in progress?")
print("status = 0:      Successful observation?")
print("status = -100:   Failed observation?")
print("status = -1000:  Severely failed observation?")
print("\nwaterfall_status = 1.0: Waterfall available?")
print("waterfall_status = 0.0: No waterfall?")
print("waterfall_status = NULL: Unknown/not processed?")

# Check if there's any documentation in the data
print("\n" + "-"*40)
print("LOOKING FOR STATUS DOCUMENTATION IN DATA:")
print("-"*40)

# Check if status values appear in other columns
status_metadata = pd.read_sql("""
    SELECT 
        status,
        waterfall_status,
        vetted_status,
        COUNT(*) as count
    FROM base_observation 
    WHERE status IN (0, 100, -100, -1000)
    GROUP BY status, waterfall_status, vetted_status
    ORDER BY status, waterfall_status
    LIMIT 20
""", engine)

print("Status combinations with vetted_status:")
print(status_metadata.to_string(index=False))


DEEP DIVE: OBSERVATION STATUS ANALYSIS
Top 15 Status Combinations:
 status  waterfall_status   count  percentage
    100               1.0 4613718       36.77
   -100               0.0 2624418       20.92
      0               NaN 2536168       20.21
    100               NaN 1770869       14.11
  -1000               NaN  946432        7.54
  -1000               0.0   53708        0.43
   -100               NaN     731        0.01
    100               0.0     142        0.00
   -100               1.0      55        0.00

----------------------------------------
STATUS CODE INTERPRETATION (Hypothesis):
----------------------------------------
status = 100:    Observation in progress?
status = 0:      Successful observation?
status = -100:   Failed observation?
status = -1000:  Severely failed observation?

waterfall_status = 1.0: Waterfall available?
waterfall_status = 0.0: No waterfall?
waterfall_status = NULL: Unknown/not processed?

----------------------------------------
LOOKING 

In [14]:
# --- Identify foreign key relationships ---
print("\n" + "="*60)
print("IDENTIFYING TABLE RELATIONSHIPS")
print("="*60)

# Based on column names, identify potential foreign keys
table_relationships = {
    'base_observation': {
        'ground_station_id': 'base_station.id',
        'sat_id': 'base_satelliteidentifier.sat_id',
        'author_id': 'base_station.owner_id? (needs verification)'
    },
    'base_station': {
        'owner_id': 'Unknown (possibly user table not in database)'
    },
    'base_satellite': {
        'satellite_entry_id': 'base_satelliteentry.id',
        'satellite_identifier_id': 'base_satelliteidentifier.id'
    },
    'base_antenna': {
        'station_id': 'base_station.id',
        'antenna_type_id': 'base_antennatype.id'
    },
    'base_frequencyrange': {
        'antenna_id': 'base_antenna.id'
    }
}

for table, relationships in table_relationships.items():
    print(f"\n{table}:")
    for fk_col, ref_table in relationships.items():
        # Check if column exists in table
        try:
            df_check = pd.read_sql(f"SELECT {fk_col} FROM {table} LIMIT 1", engine)
            print(f"  ✓ {fk_col} -> {ref_table}")
        except:
            print(f"  ✗ {fk_col} (column might not exist)")


IDENTIFYING TABLE RELATIONSHIPS

base_observation:
  ✓ ground_station_id -> base_station.id
  ✓ sat_id -> base_satelliteidentifier.sat_id
  ✓ author_id -> base_station.owner_id? (needs verification)

base_station:
  ✓ owner_id -> Unknown (possibly user table not in database)

base_satellite:
  ✓ satellite_entry_id -> base_satelliteentry.id
  ✓ satellite_identifier_id -> base_satelliteidentifier.id

base_antenna:
  ✓ station_id -> base_station.id
  ✓ antenna_type_id -> base_antennatype.id

base_frequencyrange:
  ✓ antenna_id -> base_antenna.id


In [15]:
# --- Summary statistics for numerical columns in observations ---
print("\n" + "="*60)
print("NUMERICAL COLUMNS SUMMARY IN OBSERVATIONS")
print("="*60)

# Get summary for columns that might have data
numerical_cols = ['max_altitude', 'rise_azimuth', 'set_azimuth', 'station_alt', 'station_lat', 'station_lng']

summary_queries = []
for col in numerical_cols:
    summary_queries.append(f"""
        '{col}' as column_name,
        COUNT({col}) as non_null_count,
        ROUND(COUNT({col}) * 100.0 / COUNT(*), 2) as non_null_percentage,
        ROUND(AVG({col}), 4) as avg_value,
        ROUND(MIN({col}), 4) as min_value,
        ROUND(MAX({col}), 4) as max_value,
        ROUND(STD({col}), 4) as std_dev
    """)

summary_query = f"""
SELECT {', '.join(summary_queries)}
FROM base_observation
WHERE 1=1
"""

try:
    numerical_summary = pd.read_sql(summary_query, engine)
    print("Summary of numerical columns (sample):")
    for i, col in enumerate(numerical_cols):
        row = numerical_summary.iloc[i]
        print(f"\n{col}:")
        print(f"  Non-NULL: {row['non_null_percentage']}% ({row['non_null_count']:,} rows)")
        if row['non_null_percentage'] > 0:
            print(f"  Avg: {row['avg_value']}, Min: {row['min_value']}, Max: {row['max_value']}")
except Exception as e:
    print(f"Could not compute numerical summaries: {e}")
    # Fallback to simpler query
    for col in numerical_cols[:3]:  # Just check first few
        simple_check = pd.read_sql(f"""
            SELECT COUNT({col}) as non_null_count 
            FROM base_observation 
            WHERE {col} IS NOT NULL
        """, engine)
        print(f"{col}: {simple_check['non_null_count'][0]:,} non-NULL rows")


NUMERICAL COLUMNS SUMMARY IN OBSERVATIONS
Summary of numerical columns (sample):

max_altitude:
Could not compute numerical summaries: unsupported format string passed to Series.__format__
max_altitude: 12,544,939 non-NULL rows
rise_azimuth: 12,544,939 non-NULL rows
set_azimuth: 12,544,939 non-NULL rows


In [16]:
# --- Data Quality Recommendations ---
print("\n" + "="*60)
print("DATA QUALITY RECOMMENDATIONS FOR ML PIPELINE")
print("="*60)

recommendations = [
    ("CRITICAL", "Define target variable", 
     "Need to map status codes (0, 100, -100, -1000) to success/failure labels"),
    
    ("HIGH", "Handle missing station data", 
     "station_lat, station_lng, station_alt are 100% NULL in observation table. \
     Need to join with base_station table"),
    
    ("HIGH", "Investigate status code meanings", 
     "Status codes need interpretation. Check: status=0 with waterfall_status=NULL might be successful"),
    
    ("MEDIUM", "Calculate derived features", 
     "Compute: duration = end - start, time_of_day from start, day_of_week"),
    
    ("MEDIUM", "Clean waterfall_status", 
     "Has both numeric (0.0, 1.0) and NULL values. Convert to categorical"),
    
    ("LOW", "Investigate vetted_status", 
     "Column exists but not explored. Might provide ground truth labels")
]

for priority, item, details in recommendations:
    print(f"\n[{priority}] {item}")
    print(f"   {details}")


DATA QUALITY RECOMMENDATIONS FOR ML PIPELINE

[CRITICAL] Define target variable
   Need to map status codes (0, 100, -100, -1000) to success/failure labels

[HIGH] Handle missing station data
   station_lat, station_lng, station_alt are 100% NULL in observation table.      Need to join with base_station table

[HIGH] Investigate status code meanings
   Status codes need interpretation. Check: status=0 with waterfall_status=NULL might be successful

[MEDIUM] Calculate derived features
   Compute: duration = end - start, time_of_day from start, day_of_week

[MEDIUM] Clean waterfall_status
   Has both numeric (0.0, 1.0) and NULL values. Convert to categorical

[LOW] Investigate vetted_status
   Column exists but not explored. Might provide ground truth labels


In [17]:
# --- Save enhanced schema documentation ---
import json
from datetime import datetime

# Create comprehensive schema document
schema_document = {
    "analysis_date": datetime.now().isoformat(),
    "database": DB_NAME,
    "tables": {},
    "key_findings": {
        "observation_status_codes": {
            "100": "Most common (46.1M rows) - possibly 'in progress'",
            "-100": "2nd most common (26.2M rows) - possibly 'failed'",
            "0": "2.5M rows - possibly 'successful'",
            "-1000": "1M rows - possibly 'severely failed'"
        },
        "data_quality_issues": [
            "station_* columns 100% NULL in observations (need join)",
            "Many transmitter_* columns mostly NULL",
            "Need to interpret status codes"
        ],
        "recommended_target_variable": "Map status=0 as success, others as failure (to be validated)"
    }
}

# Add table details
for summary in schema_summaries:
    table_name = summary["table"]
    schema_document["tables"][table_name] = {
        "row_count": int(summary["row_count"]),
        "columns": summary["columns"],
        "preview_sample": summary["preview"].head(3).to_dict(orient='records')
    }

# Save as JSON
with open("enhanced_schema_document.json", "w") as f:
    json.dump(schema_document, f, indent=2, default=str)

print("\n📊 Enhanced schema documentation saved to 'enhanced_schema_document.json'")


📊 Enhanced schema documentation saved to 'enhanced_schema_document.json'


In [18]:

# Save to Markdown with fallback
import importlib 
with open("schema_summary.md", "w", encoding="utf-8") as f:
    f.write("# 📊 SatNOGS Dataset Schema Summary\n\n")
    for s in schema_summaries:
        f.write(f"## Table: {s['table']}\n")
        f.write(f"- Row count: {s['row_count']}\n")
        f.write(f"- Columns: {', '.join(s['columns'])}\n\n")
        f.write("### Preview (first 5 rows)\n")
        
        # Try to use to_markdown if tabulate is available
        try:
            importlib.import_module("tabulate")
            f.write(s["preview"].head().to_markdown(index=False))
        except ImportError:
            # Fallback to plain text if tabulate is missing
            f.write(s["preview"].head().to_string(index=False))
        
        f.write("\n\n---\n\n")
        print(f"📄 Saved preview for {s['table']}")


📄 Saved preview for base_antenna
📄 Saved preview for base_antennatype
📄 Saved preview for base_frequencyrange
📄 Saved preview for base_launch
📄 Saved preview for base_mode
📄 Saved preview for base_observation
📄 Saved preview for base_operator
📄 Saved preview for base_satellite
📄 Saved preview for base_satelliteentry
📄 Saved preview for base_satelliteidentifier
📄 Saved preview for base_station
📄 Saved preview for base_stationstatuslog
📄 Saved preview for base_stationtype
📄 Saved preview for base_telemetry
📄 Saved preview for base_transmitterentry
